# CURRENCY CALCULATION

In [184]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Destiny currency scraping


The currency exchange between the departure country and the arrival country will be calculated from the web OANDA('https://www.xe.com/es/currencyconverter/'). The data will be scraped with selenium. From the web Currency-ISO (https://www.currency-iso.org/en/home/tables/table-a1.html) we will get downloaded an xls file with the currency ISO code of each country. We have downloaded it and save as .csv

First of all, we will read the currency ISO codes. 

In [217]:
#Read the currency.csv file

currency = pd.read_csv('data/currency.csv')
currency.head()

,ENTITY,Currency,Alphabetic Code,Numeric Code,Minor unit,Fund
0,AFGHANISTAN,Afghani,AFN,971.0,2,NaN
1,ÅLAND ISLANDS,Euro,EUR,978.0,2,NaN
2,ALBANIA,Lek,ALL,8.0,2,NaN
3,ALGERIA,Algerian Dinar,DZD,12.0,2,NaN
4,AMERICAN SAMOA,US Dollar,USD,840.0,2,NaN


We should clean the dataframe currency (ENTITY column) as the data would be compare and must be similar.

In [218]:
#The ENTITY (country) is given in capital letters, so we shoul change it to capitalize.

def capit(x):
    return x.title()

currency['ENTITY'] = currency['ENTITY'].apply(capit)
currency.head()

,ENTITY,Currency,Alphabetic Code,Numeric Code,Minor unit,Fund
0,Afghanistan,Afghani,AFN,971.0,2,NaN
1,Åland Islands,Euro,EUR,978.0,2,NaN
2,Albania,Lek,ALL,8.0,2,NaN
3,Algeria,Algerian Dinar,DZD,12.0,2,NaN
4,American Samoa,US Dollar,USD,840.0,2,NaN


In [219]:
#Delete parts between brackets
def no_brackets(entity):
    result = re.sub('\(.*?\)', '', entity)
    return result

currency['ENTITY'] = currency['ENTITY'].apply(no_brackets)
currency.head(30)


,ENTITY,Currency,Alphabetic Code,Numeric Code,Minor unit,Fund
0,Afghanistan,Afghani,AFN,971.0,2,NaN
1,Åland Islands,Euro,EUR,978.0,2,NaN
2,Albania,Lek,ALL,8.0,2,NaN
3,Algeria,Algerian Dinar,DZD,12.0,2,NaN
4,American Samoa,US Dollar,USD,840.0,2,NaN
5,Andorra,Euro,EUR,978.0,2,NaN
6,Angola,Kwanza,AOA,973.0,2,NaN
7,Anguilla,East Caribbean Dollar,XCD,951.0,2,NaN
8,Antarctica,No universal currency,NaN,NaN,NaN,NaN
9,Antigua And Barbuda,East Caribbean Dollar,XCD,951.0,2,NaN


In [220]:
#Now we drop duplicates
before = len(currency)
currency_clean = currency.drop_duplicates(subset=['ENTITY'])
after = len(currency_clean)
print(f'Number of duplicate records dropped: {before - after}')

Number of duplicate records dropped: 19


In [222]:
#Clean if there are any space
def no_spaces(string):
    clean = string.strip()
    return clean

In [223]:
currency_clean['ENTITY'] = currency_clean['ENTITY'].apply(no_spaces)

<ipython-input-223-27eefdf695b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_clean['ENTITY'] = currency_clean['ENTITY'].apply(no_spaces)


In [224]:
currency.to_csv('data/currency_table.csv',index=False)

In [225]:
currency.dtypes

ENTITY              object
Currency            object
Alphabetic Code     object
Numeric Code       float64
Minor unit          object
Fund                object
dtype: object

From the departure_airport_df and arrival_airport_df we get the country name, thus looking for that country in our currency.csv we can get the currency code

In [226]:
currency = pd.read_csv('data/currency_table.csv')
currency.head()

,ENTITY,Currency,Alphabetic Code,Numeric Code,Minor unit,Fund
0,Afghanistan,Afghani,AFN,971.0,2,NaN
1,Åland Islands,Euro,EUR,978.0,2,NaN
2,Albania,Lek,ALL,8.0,2,NaN
3,Algeria,Algerian Dinar,DZD,12.0,2,NaN
4,American Samoa,US Dollar,USD,840.0,2,NaN


In [227]:
#Example
departure_country = 'Turkey'
arrival_country = 'Netherlands'

In [228]:
#Get currency code of each country
departure_curr_df = currency.loc[currency['ENTITY'] == departure_country]
departure_curr_code = departure_curr_df['Alphabetic Code'].values[0]
print(f'Departure currency code is: {departure_curr_code}')

arrival_curr_df = currency.loc[currency['ENTITY'] == arrival_country]
arrival_curr_code = arrival_curr_df['Alphabetic Code'].values[0]
arrival_curr_name = arrival_curr_df['Currency'].values[0]
print(f'Arrival currency code is: {arrival_curr_code}')

Departure currency code is: TRY


IndexError: index 0 is out of bounds for axis 0 with size 0

We should scrap the currency exchange web applying these codes


In [186]:
departure_curr_code = 'JPY'
arrival_curr_code = 'EUR'

In [187]:
#Get driver
driver = webdriver.Chrome('/home/viki/Bootcamp/drivers/chromedriver')
driver.get('https://www.xe.com/es/currencyconverter/')

In [188]:
#Close cookies
cookies_button = driver.find_element_by_xpath("//button[@class='button__BaseButton-sc-1qpsalo-0 ctapkr']")
driver.execute_script("arguments[0].click();", cookies_button)

In [189]:
#Input departure currency code
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "midmarketFromCurrency")))
first_box = driver.find_element_by_id('midmarketFromCurrency')
first_box.send_keys(departure_curr_code)
first_box.send_keys(Keys.ENTER)


In [190]:
#Input arrival currency code
second_box = driver.find_element_by_id('midmarketToCurrency')
second_box.send_keys(arrival_curr_code)
second_box.send_keys(Keys.ENTER)

In [191]:
#Click on submit button
submit_button = driver.find_element_by_xpath("//button[@type='submit']")
driver.execute_script("arguments[0].click();", submit_button)

In [192]:
#Take conversion rule
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "unit-rates___StyledDiv-sc-1dk593y-0.dEqdnx")))
rule = driver.find_element_by_class_name("unit-rates___StyledDiv-sc-1dk593y-0.dEqdnx").text
rule

'1 EUR = 131,479 JPY'

In [193]:
#Print results
print (f'The currency in {arrival_country} is {arrival_curr_name}')
print(f"And don't forget: {rule}")


The currency in Italy is Euro
And don't forget: 1 EUR = 131,479 JPY
